In [1]:
import pandas as pd

In [2]:
data_path = './corona_scotland.xlsx'

# Cases and Death Data

In [3]:
df = pd.read_excel(data_path, 
                   dtype={'Total Tests': 'Int32',
                          'Total Positive Cases': 'Int32',
                          'Total Deaths': 'Int32'}
                   ).ffill()
df = df[df['Health Board'] != 'Golden Jubilee National Hospital']

In [4]:
df

,Date,Health Board,Number of Positive cases,Total Tests,Total Positive Cases,Total Deaths
0,2020-03-06,Tayside,1,1514,11,0
1,2020-03-06,Ayrshire & Arran,1,1514,11,0
2,2020-03-06,Forth Valley,2,1514,11,0
3,2020-03-06,Greater Glasgow and Clyde,1,1514,11,0
4,2020-03-06,Grampian,3,1514,11,0
...,...,...,...,...,...,...
826,2020-05-07,Lothian,2241,67097,12924,1762
827,2020-05-07,Orkney,7,67097,12924,1762
828,2020-05-07,Shetland,54,67097,12924,1762
829,2020-05-07,Tayside,1465,67097,12924,1762


## clean data

In [5]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        63
Lothian                          63
Tayside                          63
Grampian                         63
Fife                             63
Forth Valley                     63
Lanarkshire                      62
Shetland                         60
Borders                          58
Highland                         55
Dumfries and Galloway            53
Ayrshire and Arran               40
Orkney                           37
Eileanan Siar (Western Isles)    37
Ayrshire and Arran               20
Ayrshire & Arran                  3
Name: Health Board, dtype: int64

In [6]:
# clean up some names
df.loc[df['Health Board'] == 'Ayrshire\xa0and Arran', 'Health Board'] = 'Ayrshire and Arran' 
df.loc[df['Health Board'] == 'Ayrshire & Arran', 'Health Board'] = 'Ayrshire and Arran' 

In [7]:
df['Health Board'].value_counts()

Greater Glasgow and Clyde        63
Lothian                          63
Ayrshire and Arran               63
Tayside                          63
Grampian                         63
Fife                             63
Forth Valley                     63
Lanarkshire                      62
Shetland                         60
Borders                          58
Highland                         55
Dumfries and Galloway            53
Orkney                           37
Eileanan Siar (Western Isles)    37
Name: Health Board, dtype: int64

## table of number cases per region over time

In [8]:
df_cases = df[['Date', 'Health Board', 'Number of Positive cases']]

In [9]:
df_table = pd.pivot_table(df_cases, 
                          columns=['Health Board'], 
                          values=['Number of Positive cases'], 
                          index=['Date'],
                          fill_value=0,
                         ).astype('Int32')

df_table.columns = df_table.columns.droplevel(0)

In [10]:
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [11]:
df_table.to_pickle('corona_scotland_regions_ts.pk')

## table of national cases, tests, and deaths over time

In [12]:
df_national = (df.drop(columns=['Health Board', 'Number of Positive cases'])
                 .groupby('Date').first()
              )

In [13]:
df_national.index

DatetimeIndex(['2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
               '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
               '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17',
               '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21',
               '2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
               '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
               '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
               '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06',
               '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10',
               '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14',
               '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18',
               '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22',
               '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26',
               '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30',
      

In [14]:
df_national.to_pickle('corona_scotland_total_stats.pk')

## Scotland Population Data

In [15]:
region_pops = {
    'Ayrshire and Arran': 369670,
    'Borders': 115270, 
    'Dumfries and Galloway': 148790,
    'Fife': 371910,
    'Forth Valley': 306070,
    'Grampian': 584550,
    'Greater Glasgow and Clyde': 1196335,
    'Highland': 321800,
    'Lanarkshire': 563185,
    'Lothian': 897770,
    'Orkney': 22190,
    'Shetland': 22990,
    'Tayside': 416080,
    'Eileanan Siar (Western Isles)': 26500,
    #'Scotland (Total)': 5438000,
}

In [16]:
df_pop = pd.DataFrame(region_pops.items(), columns=['Region', 'Pop']).set_index('Region').sort_index()

In [17]:
df_pop.loc['Scotland (Total)'] = 5438000

In [18]:
df_pop

,Pop
Region,
Ayrshire and Arran,369670
Borders,115270
Dumfries and Galloway,148790
Eileanan Siar (Western Isles),26500
Fife,371910
Forth Valley,306070
Grampian,584550
Greater Glasgow and Clyde,1196335
Highland,321800


In [19]:
df_pop[:14].sum()
## hmm missing few thousands??

Pop    5363110
dtype: int64

In [20]:
df_pop.to_pickle('scotland_population.pk')

# Hospital Data

In [21]:
from datetime import datetime

In [31]:
df = pd.read_excel(data_path, 
                   usecols=['date', 'health_board', 
                            'num_hospital', 'num_icu'],
                   dtype={
                       'date': datetime,
                       'health_board': 'str',
                       'num_hospital': 'Int32',
                       'num_icu': 'Int32'},
                   na_values={'num_hospital': '*',
                              'num_icu': '*'},
                   sheet_name=1,
                   )
df['date'].ffill(inplace=True)
df.fillna(0, inplace=True)

In [32]:
df

,date,health_board,num_hospital,num_icu
0,2020-04-08,Ayrshire and Arran,91,16
1,2020-04-08,Borders,56,9
2,2020-04-08,Dumfries and Galloway,44,6
3,2020-04-08,Fife,127,13
4,2020-04-08,Forth Valley,107,9
...,...,...,...,...
444,2020-05-07,Orkney,0,0
445,2020-05-07,Shetland,0,0
446,2020-05-07,Tayside,65,0
447,2020-05-07,Eileanan Siar (Western Isles),0,0


In [33]:
df_hos = df[['date', 'health_board', 'num_hospital']]
df_icu = df[['date', 'health_board', 'num_icu']]

In [34]:
df_table = pd.pivot_table(df_hos, 
                          columns=['health_board'], 
                          values=['num_hospital'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [35]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-03,115,35,18,0,123,66,0,147,526,41,233,269,0,0,88,1661
2020-05-04,86,30,21,0,125,70,5,130,533,44,257,327,0,0,92,1720
2020-05-05,93,32,21,0,118,81,0,165,523,52,223,274,0,0,64,1646
2020-05-06,108,37,17,0,109,82,0,151,509,37,229,274,0,0,64,1617
2020-05-07,97,38,0,0,100,79,0,135,506,27,212,302,0,0,65,1561


In [36]:
df_table.to_pickle('corona_scotland_regions_hospital_ts.pk')

In [37]:
df_table = pd.pivot_table(df_icu, 
                          columns=['health_board'], 
                          values=['num_icu'], 
                          index=['date'],
                          fill_value=0,
                         ).astype('int32')

df_table.columns = df_table.columns.droplevel(0)
df_table['Scotland (Total)'] = df_table.sum(axis=1)

In [38]:
df_table.tail()

health_board,Ayrshire and Arran,Borders,Dumfries and Galloway,Eileanan Siar (Western Isles),Fife,Forth Valley,Golden Jubilee National Hospital,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Scotland (Total)
date,,,,,,,,,,,,,,,,
2020-05-03,8,0,0,0,6,0,0,13,39,0,8,12,0,0,0,86
2020-05-04,6,0,0,0,6,6,0,13,39,0,7,14,0,0,0,91
2020-05-05,8,0,0,0,5,0,0,13,32,0,10,18,0,0,0,86
2020-05-06,7,0,0,0,0,0,0,11,30,0,5,18,0,0,0,71
2020-05-07,7,0,0,0,0,0,0,11,32,0,7,12,0,0,0,69


In [39]:
df_table.to_pickle('corona_scotland_regions_icu_ts.pk')